In [2]:
import geopandas as gpd
import pandas as pd

# **Quelle est la surface des espaces naturelles en Martinique ?** 

* *Données :* OCS-GE V2 Martinique
* *Bibliographie:* https://www.picto-occitanie.fr/upload/gedit/1/ocsge_occitanie/FicheGuide_groupeC_v3.3.pdf

In [42]:
code_cs = pd.read_csv("ocsge-V2_CS-labels.csv", sep=';', encoding='utf-8')
code_us = pd.read_csv("ocsge-V2_US-labels.csv", sep=';', encoding='utf-8')

code_cs_wide = pd.read_csv("ocsge-V2_CS-labels-wide.csv", sep=';', encoding='utf-8')
code_us_wide = pd.read_csv("ocsge-V2_US-labels-wide.csv", sep=';', encoding='utf-8')

In [67]:
headers = ['code_lvl6'] + [f"label_lvl{j}" for j in [1,2,3,6]]
headers

['code_lvl6', 'label_lvl1', 'label_lvl2', 'label_lvl3', 'label_lvl6']

In [68]:
code_cs_wide.loc[:,headers]

,code_lvl6,label_lvl1,label_lvl2,label_lvl3,label_lvl6
0,CS1.1.1.1,Sans végétation,Surfaces anthropisées,Zones imperméables,Zones bâties
1,CS1.1.1.2,Sans végétation,Surfaces anthropisées,Zones imperméables,Zones non bâties
2,CS1.1.2.1,Sans végétation,Surfaces anthropisées,Zones perméables,Zones à matériaux minéraux
3,CS1.1.2.2,Sans végétation,Surfaces anthropisées,Zones perméables,Zones à autres matériaux composites
4,CS1.2.1,Sans végétation,Surfaces naturelles,Sols nus,Sols nus
5,CS1.2.1.6,Sans végétation,Surfaces naturelles,Sols nus,Etang bois sec
6,CS1.2.2,Sans végétation,Surfaces naturelles,Surfaces d'eau,Surfaces d'eau
7,CS2.1.1.1,Avec végétation,Végétation ligneuse,Formations arborées,Peuplement de feuillus
8,CS2.1.1.1.21,Avec végétation,Végétation ligneuse,Formations arborées,Mangrove
9,CS2.1.1.1.22,Avec végétation,Végétation ligneuse,Formations arborées,Forêt littorale


In [57]:
pd.options.display.max_columns = None

pd.merge(code_cs_wide.loc[:,[headers]], percent, how='outer', right_index=True, left_on=['code_lvl6','label_lvl6']).sort_index()

KeyError: "None of [Index([('code_lvl6', 'label_lvl1', 'label_lvl2', 'label_lvl3', 'label_lvl4', 'label_lvl5', 'label_lvl6')], dtype='object')] are in the [columns]"

In [4]:
PATH = "Telechargement_1670000423_5587/deal972_occupation_sol_ge_2017_v2_s.shp"

In [5]:
gdf = gpd.read_file(PATH, encoding='utf-8')
gdf = pd.merge(gdf, code_cs, left_on='code_cs', right_on='code').rename({'label':'label_CS'}, axis=1).drop('code', axis=1)
gdf = pd.merge(gdf, code_us, left_on='code_us', right_on='code').rename({'label':'label_US'}, axis=1).drop('code', axis=1)
gdf['area'] = gdf.area

In [6]:
gdf.head()

,gid,id,code_cs,code_us,millesime,source,ossature,id_origine,code_or,geometry,label_CS,label_US,area
0,1,1,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((727567.750 1610187.276, 727567.770 1...",Zones bâties,Agriculture,251.073044
1,2,2,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((695732.002 1632771.602, 695751.802 1...",Zones bâties,Agriculture,1766.216504
2,3,3,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((696053.251 1627147.354, 696053.861 1...",Zones bâties,Agriculture,201.015439
3,4,4,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((696054.051 1627062.104, 696051.101 1...",Zones bâties,Agriculture,2375.714140
4,5,5,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((696799.001 1626255.394, 696775.401 1...",Zones bâties,Agriculture,611.276706


In [7]:
gdf.crs

<Derived Projected CRS: EPSG:5490>
Name: RGAF09 / UTM zone 20N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: French Antilles onshore and offshore west of 60°W - Guadeloupe (including Grande Terre, Basse Terre, Marie Galante, Les Saintes, Iles de la Petite Terre, La Desirade); Martinique; St Barthélemy; northern St Martin.
- bounds: (-63.66, 14.08, -60.0, 18.31)
Coordinate Operation:
- name: UTM zone 20N
- method: Transverse Mercator
Datum: Reseau Geodesique des Antilles Francaises 2009
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [8]:
TYPOLOGIE = [
# Surfaces d'eau
 ('CS1.2.2','US1.1'),
 ('CS1.2.2','US1.2'),
 ('CS1.2.2','235'),
 ('CS1.2.2','US4.1'), # Sous-code
 ('CS1.2.2','US6.3'),
 ('CS1.2.2','US6.4'),
 
 # Peuplements de feuillus
 ('CS2.1.1.1','US1.3'),
 ('CS2.1.1.1','235'),
 ('CS2.1.1.1','US6.3'),

# Peuplements de conifères
 ('CS2.1.1.2','US1.3'),
 ('CS2.1.1.2','US235'),
 ('CS2.1.1.2','US6.3'),

# Peuplements mixtes
 ('CS2.1.1.3','US1.3'),
 ('CS2.1.1.3','235'),
 ('CS2.1.1.3','US6.3'),
 
# Formations arbustives et sous-arbrisseaux
 ('CS2.1.2','US1.3'),
 ('CS2.1.2','235'),
 ('CS2.1.2','US6.3'),
 ('CS2.1.2','US6.4'),
 
# Formations arbustives et sous-arbrisseaux
 ('CS2.2.1','US1.3'),
 ('CS2.2.1','US6.3'),
 ('CS2.2.1','US6.4'),

# Autres formations non-ligneuses
 ('CS2.2.2','US6.3'),
 ]

In [9]:
# s = '(\n'
# for cs, us in TYPOLOGIE:
#     s += f" | (gdf.code_cs.str.contains('{cs}') & gdf.code_us.str.contains('{us}')) \n"
# s += ')'
# print(s)

mask_typolgie = (
 (gdf.code_cs.str.contains('CS1.2.2') & gdf.code_us.str.contains('US1.1')) 
 | (gdf.code_cs.str.contains('CS1.2.2') & gdf.code_us.str.contains('US1.2')) 
 | (gdf.code_cs.str.contains('CS1.2.2') & gdf.code_us.str.contains('235')) 
 | (gdf.code_cs.str.contains('CS1.2.2') & gdf.code_us.str.contains('US4.1')) 
 | (gdf.code_cs.str.contains('CS1.2.2') & gdf.code_us.str.contains('US6.3')) 
 | (gdf.code_cs.str.contains('CS1.2.2') & gdf.code_us.str.contains('US6.4')) 
 | (gdf.code_cs.str.contains('CS2.1.1.1') & gdf.code_us.str.contains('US1.3')) 
 | (gdf.code_cs.str.contains('CS2.1.1.1') & gdf.code_us.str.contains('235')) 
 | (gdf.code_cs.str.contains('CS2.1.1.1') & gdf.code_us.str.contains('US6.3')) 
 | (gdf.code_cs.str.contains('CS2.1.1.2') & gdf.code_us.str.contains('US1.3')) 
 | (gdf.code_cs.str.contains('CS2.1.1.2') & gdf.code_us.str.contains('235')) 
 | (gdf.code_cs.str.contains('CS2.1.1.2') & gdf.code_us.str.contains('US6.3')) 
 | (gdf.code_cs.str.contains('CS2.1.1.3') & gdf.code_us.str.contains('US1.3')) 
 | (gdf.code_cs.str.contains('CS2.1.1.3') & gdf.code_us.str.contains('235')) 
 | (gdf.code_cs.str.contains('CS2.1.1.3') & gdf.code_us.str.contains('US6.3')) 
 | (gdf.code_cs.str.contains('CS2.1.2') & gdf.code_us.str.contains('US1.3')) 
 | (gdf.code_cs.str.contains('CS2.1.2') & gdf.code_us.str.contains('235')) 
 | (gdf.code_cs.str.contains('CS2.1.2') & gdf.code_us.str.contains('US6.3')) 
 | (gdf.code_cs.str.contains('CS2.1.2') & gdf.code_us.str.contains('US6.4')) 
 | (gdf.code_cs.str.contains('CS2.2.1') & gdf.code_us.str.contains('US1.3')) 
 | (gdf.code_cs.str.contains('CS2.2.1') & gdf.code_us.str.contains('US6.3')) 
 | (gdf.code_cs.str.contains('CS2.2.1') & gdf.code_us.str.contains('US6.4')) 
 | (gdf.code_cs.str.contains('CS2.2.2') & gdf.code_us.str.contains('US6.3')) 
)

surfaces_naturelles = gdf.loc[mask_typolgie].copy()


In [10]:
pivot_gdf = gdf.pivot_table(index=['code_us','label_US'], columns=['code_cs','label_CS'], values='area', aggfunc='sum', margins=True, margins_name='TOTAL').T

In [11]:
pivot_gdf.to_csv('ocsge-V2_pivot.csv', encoding='utf-8', sep=';')

In [16]:
pivot_surface_nat = surfaces_naturelles.pivot_table(index=['code_us','label_US'], columns=['code_cs','label_CS'], values='area', aggfunc='sum', margins=True, margins_name='TOTAL').T

In [21]:
S_total=gdf.area.sum().sum()

In [29]:
percent = gdf.pivot_table(columns=['code_cs','label_CS'],
                     index=['code_us','label_US'],
                     values='area',
                     aggfunc=sum,
                     margins=True,
                     margins_name='TOTAL').divide(S_total).T

In [41]:
from IPython.display import display

df_style = (percent
            .fillna(0)
            .style.format({k:"{:.2%}" for k in percent.columns})
            .background_gradient(axis=None, vmin=0.01, vmax=0.5, cmap='viridis')
            )

df_style

,code_us,US1.1,US1.2,US1.3,US1.4,US2,US2.4.2,US2.4.3,US2.4.4,US235,US3,US4.1.1,US4.1.2,US4.1.3,US4.1.4,US4.2,US4.3,US5,US6.1,US6.2,US6.3,TOTAL
,label_US,Agriculture,Sylviculture,Activités d’extraction,Pêche et aquaculture,Production secondaire,Production d'énergie thermique,Production d'énergie biomasse,Production d'énergies renouvelables,Usage mixte,Production tertiaire,Réseaux de transport routier,Réseaux de transport ferré,Réseaux de transport aérien,Réseaux de transport fluvial et maritime,Services de logistique et de stockage,Réseaux d’utilité publique,Usage résidentiel,Zones en transition,Zones abandonnées,Sans usage,
code_cs,label_CS,,,,,,,,,,,,,,,,,,,,,
CS1.1.1.1,Zones bâties,0.07%,0.00%,0.00%,0.00%,0.14%,0.00%,0.00%,0.00%,2.92%,0.27%,0.00%,0.00%,0.01%,0.00%,0.00%,0.01%,0.80%,0.01%,0.00%,0.00%,4.23%
CS1.1.1.2,Zones non bâties,0.00%,0.00%,0.00%,0.00%,0.13%,0.01%,0.01%,0.00%,0.16%,0.21%,1.73%,0.00%,0.05%,0.05%,0.00%,0.01%,0.17%,0.00%,0.00%,0.00%,2.55%
CS1.1.2.1,Zones à matériaux minéraux,0.03%,0.00%,0.12%,0.00%,0.05%,0.00%,0.00%,0.00%,0.09%,0.02%,0.01%,0.00%,0.00%,0.00%,0.00%,0.01%,0.06%,0.07%,0.00%,0.00%,0.47%
CS1.1.2.2,Zones à autres matériaux composites,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.01%,0.00%,0.00%,0.00%,0.00%,0.01%
CS1.2.1,Sols nus,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.19%,0.19%
CS1.2.1.6,Etang bois sec,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.21%,0.21%
CS1.2.2,Surfaces d'eau,0.00%,0.00%,0.00%,0.01%,0.00%,0.00%,0.00%,0.00%,0.07%,0.00%,0.00%,0.00%,0.00%,0.10%,0.00%,0.00%,0.00%,0.00%,0.00%,0.38%,0.58%
CS2.1.1.1,Peuplement de feuillus,0.43%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.43%


In [ ]:
pivot_surface_nat.to_csv('ocsge-V2_pivot_surface-nat.csv', encoding='utf-8', sep=';')

In [ ]:
S_naturelle = surfaces_naturelles.area.sum() / 10000

print(f'Surfaces naturelles : {round(S_naturelle, 2)} ha')

Surfaces naturelles : 9506.41 ha


In [ ]:
S_total = gdf.area.sum() / 10000

print(f'Surfaces naturelles : {round(S_total, 2)} ha')

Surfaces naturelles : 110906.44 ha


In [ ]:
natural_percent = (S_naturelle / S_total) * 100

print(f'Part de surfaces naturelles : {round(natural_percent,2)} %')

Part de surfaces naturelles : 8.57 %


In [ ]:
pivot_surface_nat

,code_us,US1.1,US1.3,US235,US4.1.1,US4.1.3,US4.1.4,US6.3,TOTAL
,label_US,Agriculture,Activités d’extraction,Usage mixte,Réseaux de transport routier,Réseaux de transport aérien,Réseaux de transport fluvial et maritime,Sans usage,
code_cs,label_CS,,,,,,,,
CS1.2.2,Surfaces d'eau,54427.652337,NaN,8.184750e+05,2297.504902,36200.47382,1.075752e+06,4.219958e+06,6.207111e+06
CS2.1.1.1.21,Mangrove,NaN,NaN,NaN,NaN,NaN,NaN,2.214109e+07,2.214109e+07
CS2.1.1.1.22,Forêt littorale,NaN,NaN,NaN,NaN,NaN,NaN,4.144661e+04,4.144661e+04
CS2.1.1.1.23,Forêt sèche basse,NaN,43786.426854,6.493684e+04,NaN,NaN,NaN,7.567126e+04,1.843945e+05
CS2.1.1.1.24,Forêt sèche haute,NaN,58979.066573,1.866553e+05,NaN,NaN,NaN,1.394403e+05,3.850747e+05
CS2.1.1.1.25,Forêt moyennement humide ou humide,NaN,53295.703313,1.209544e+06,NaN,NaN,NaN,7.098041e+05,1.972644e+06
CS2.1.1.1.27,Peuplement de mahogany,NaN,NaN,2.520388e+04,NaN,NaN,NaN,2.163339e+04,4.683726e+04
CS2.1.1.1.29,Forêt marécageuse,NaN,NaN,NaN,NaN,NaN,NaN,5.773689e+02,5.773689e+02


In [ ]:
pivot_gdf

,code_us,US1.1,US1.2,US1.3,US1.4,US2,US2.4.2,US2.4.3,US2.4.4,US235,US3,...,US4.1.2,US4.1.3,US4.1.4,US4.2,US4.3,US5,US6.1,US6.2,US6.3,TOTAL
,label_US,Agriculture,Sylviculture,Activités d’extraction,Pêche et aquaculture,Production secondaire,Production d'énergie thermique,Production d'énergie biomasse,Production d'énergies renouvelables,Usage mixte,Production tertiaire,...,Réseaux de transport ferré,Réseaux de transport aérien,Réseaux de transport fluvial et maritime,Services de logistique et de stockage,Réseaux d’utilité publique,Usage résidentiel,Zones en transition,Zones abandonnées,Sans usage,
code_cs,label_CS,,,,,,,,,,,,,,,,,,,,,
CS1.1.1.1,Zones bâties,7.426252e+05,NaN,2.105905e+04,427.197370,1.517446e+06,50958.870843,519.868617,402.153998,3.238833e+07,3.020558e+06,...,NaN,6.514167e+04,3.523639e+04,1100.459409,9.760140e+04,8.852227e+06,1.174328e+05,45847.635303,NaN,4.696030e+07
CS1.1.1.2,Zones non bâties,3.013296e+04,NaN,1.489591e+04,NaN,1.468414e+06,95713.853109,74332.582136,18185.558753,1.829484e+06,2.277314e+06,...,NaN,5.695264e+05,5.786858e+05,6712.947210,1.529738e+05,1.901052e+06,2.516457e+04,2823.153486,NaN,2.825895e+07
CS1.1.2.1,Zones à matériaux minéraux,3.291373e+05,NaN,1.313049e+06,NaN,5.197642e+05,8268.868218,NaN,NaN,1.047444e+06,2.465924e+05,...,13101.827212,8.014760e+03,4.146834e+04,17900.641713,1.366498e+05,6.523445e+05,8.160238e+05,8898.616853,3.109537e+04,5.247119e+06
CS1.1.2.2,Zones à autres matériaux composites,NaN,NaN,7.833140e+03,NaN,1.911080e+04,NaN,NaN,NaN,1.024503e+04,NaN,...,NaN,NaN,NaN,NaN,1.267051e+05,NaN,NaN,NaN,NaN,1.638941e+05
CS1.2.1,Sols nus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.157113e+06,2.157113e+06
CS1.2.1.6,Etang bois sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.353100e+06,2.353100e+06
CS1.2.2,Surfaces d'eau,5.442765e+04,NaN,2.642268e+04,86800.182748,1.822466e+03,NaN,NaN,NaN,8.184750e+05,4.747272e+04,...,NaN,3.620047e+04,1.075752e+06,NaN,3.802106e+04,1.724197e+04,NaN,NaN,4.219958e+06,6.424892e+06
CS2.1.1.1,Peuplement de feuillus,4.807609e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.807609e+06
